In [1]:
import sqlite3
import pandas as pd
from collections import Counter

# ——————————————————————————
# 1) Conexión a la base de datos SQLite
# ——————————————————————————
DB_PATH = "data/AmITheAsshole.sqlite"   # ajústalo a donde esté tu archivo .sqlite
conn = sqlite3.connect(DB_PATH)

# ——————————————————————————
# 2) Carga de tablas
# ——————————————————————————
df_submissions = pd.read_sql_query(
    "SELECT submission_id, title, selftext FROM submission;",
    conn
)
df_comments = pd.read_sql_query(
    "SELECT submission_id, message FROM comment;",
    conn
)
conn.close()

# ——————————————————————————
# 3) Inferencia de etiqueta (verdict) desde los mensajes
# ——————————————————————————
verdict_keywords = ['NTA', 'YTA', 'ESH', 'NAH', 'INFO']

def extract_verdict(msg: str) -> str:
    msg = msg.upper()
    for word in verdict_keywords:
        # busca la palabra exacta en el split
        if word in msg.split():
            return word
    return None

# aplica y filtra solo comentarios que contengan una etiqueta válida
df_comments['label'] = df_comments['message'].apply(extract_verdict)
df_comments = df_comments[df_comments['label'].notnull()]

# ——————————————————————————
# 4) Para cada post, quedarnos con la etiqueta más frecuente
# ——————————————————————————
submission_verdicts = (
    df_comments
    .groupby('submission_id')['label']
    .apply(lambda x: Counter(x).most_common(1)[0][0])
    .reset_index()
)

# ——————————————————————————
# 5) Merge con los submissions y creación de la columna 'text'
# ——————————————————————————
df_merged = df_submissions.merge(submission_verdicts, on='submission_id')
df_merged['text'] = (
    df_merged['title'].fillna("") + " " +
    df_merged['selftext'].fillna("")
)

# ——————————————————————————
# 6) Tu dataset final listo para guardar o procesar
# ——————————————————————————
df_final = df_merged.rename(columns={'label':'gold_label'})
# si quieres la columna como 'label' simplemente omite el rename:
# df_final = df_merged

print(df_final.head())
print(f"Total posts extraídos: {len(df_final)}")

# (Opcional) exporta a CSV para no tener que volver a ejecutar:
df_final.to_csv("aita_final_dataset.csv", index=False)

  submission_id                                              title  \
0        xt1ksm            AITA Monthly Open Forum Spooktober 2022   
1        yiplwk  AITA for asking my friend to move a picture of...   
2        yiv572  AITA for asking my husband to stay with me whi...   
3        yimgaf  AITA for telling my SIL to stop talking about ...   
4        yin7pf  AITA for wanting to meet my "daughter" after g...   

                                            selftext gold_label  \
0  #Keep things civil. Rules still apply.\n\n##Th...        NTA   
1  \n\nMe (M32) and my wife, Dahlia (F28) lost ou...        YTA   
2  Throwaway my family knows my account. I'll get...        YTA   
3  My (37M) wife (37F) is pregnant with our first...        NTA   
4  Long story short: in my (40f) twenties I had a...        YTA   

                                                text  
0  AITA Monthly Open Forum Spooktober 2022 #Keep ...  
1  AITA for asking my friend to move a picture of...  
2  AITA for

In [2]:
df_final.head(10)

,submission_id,title,selftext,gold_label,text
0,xt1ksm,AITA Monthly Open Forum Spooktober 2022,#Keep things civil. Rules still apply.\n\n##Th...,NTA,AITA Monthly Open Forum Spooktober 2022 #Keep ...
1,yiplwk,AITA for asking my friend to move a picture of...,"\n\nMe (M32) and my wife, Dahlia (F28) lost ou...",YTA,AITA for asking my friend to move a picture of...
2,yiv572,AITA for asking my husband to stay with me whi...,Throwaway my family knows my account. I'll get...,YTA,AITA for asking my husband to stay with me whi...
3,yimgaf,AITA for telling my SIL to stop talking about ...,My (37M) wife (37F) is pregnant with our first...,NTA,AITA for telling my SIL to stop talking about ...
4,yin7pf,"AITA for wanting to meet my ""daughter"" after g...",Long story short: in my (40f) twenties I had a...,YTA,"AITA for wanting to meet my ""daughter"" after g..."
5,yipi15,AITA for making my roommate replace my garlic ...,"Last week, I came home and my roommate told me...",YTA,AITA for making my roommate replace my garlic ...
6,yihz2z,AITA for refusing to give my son pants?,Today kids can wear their costumes to school. ...,NTA,AITA for refusing to give my son pants? Today ...
7,yinf25,AITA for ditching the wedding my parents paid ...,"The more I think about it, the more I feel bad...",NTA,AITA for ditching the wedding my parents paid ...
8,yiczsa,AITA for refusing to be my sister's MOH after ...,I (33f) have been with my bf (35m) for over 10...,NTA,AITA for refusing to be my sister's MOH after ...
9,yir4uf,AITA for “revealing” my pregnancy at a wedding?,I (27f) have been dating my boyfriend (28m) fo...,NTA,AITA for “revealing” my pregnancy at a wedding...


In [6]:
from openai import AzureOpenAI
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from dotenv import load_dotenv

# ——————————————————————————
# 1) Configuración de AzureOpenAI
# ——————————————————————————
load_dotenv()  # Cargar las variables desde .env

deployment_name = "gpt"

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", "https://invuniandesai-2.openai.azure.com/"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-10-21"
)

# ——————————————————————————
# 2) Función de clasificación AITA con manejo de errores
# ——————————————————————————
def clasifica_aita(texto: str, index: int = None) -> str:
    try:
        respuesta = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content":
                 "Eres un clasificador de etiquetas para el subreddit AITA. "
                 "Responde únicamente con UNA de estas etiquetas: YTA, NTA, NAH, ESH o INFO."},
                {"role": "user", "content": texto}
            ],
            temperature=0.8
        )
        etiqueta = respuesta.choices[0].message.content.strip()
        return etiqueta
    except Exception as e:
        print(f"⚠️ Entrada con error en índice {index}: {e}")
        return "ERROR"

# ——————————————————————————
# 3) Procesamiento del DataFrame con logging de errores
# ——————————————————————————
df = pd.read_csv("aita_final_dataset.csv")
predicciones = []

for idx, row in df.iterrows():
    pred = clasifica_aita(row["text"], index=idx)
    predicciones.append(pred)

df["pred_openai"] = predicciones

# ——————————————————————————
# 4) Cálculo de métricas (excluyendo errores si es necesario)
# ——————————————————————————
df_valid = df[df["pred_openai"] != "ERROR"]
y_true = df_valid["gold_label"]
y_pred = df_valid["pred_openai"]
labels = ["YTA", "NTA", "NAH", "ESH", "INFO"]

print("=== Métricas Azure OpenAI ===")
print(f"Accuracy: {accuracy_score(y_true, y_pred):.2%}\n")
print(classification_report(y_true, y_pred, labels=labels, digits=4))
print("Matriz de confusión:\n", confusion_matrix(y_true, y_pred, labels=labels))

# ——————————————————————————
# 5) Guardar resultados
# ——————————————————————————
df[["submission_id", "gold_label", "pred_openai"]] \
    .to_csv("aita_openai_results.csv", index=False)

# ——————————————————————————
# 6) Reporte de errores
# ——————————————————————————
errores = df[df["pred_openai"] == "ERROR"]
print(f"\nTotal de entradas con error: {len(errores)}")
if not errores.empty:
    errores.to_csv("aita_errores_filtrados.csv", index=False)
    print("Errores guardados en 'aita_errores_filtrados.csv'")

⚠️ Entrada con error en índice 127: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}
⚠️ Entrada con error en índice 131: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our con